# CUSTOMER CHURN IN TELECOMUNICATION

<img src="https://media.licdn.com/dms/image/C5612AQH0DqxxGFkCvQ/article-cover_image-shrink_720_1280/0/1626825664542?e=1715212800&v=beta&t=-3wL6onvoFSlGgxRkn8ZH5rXP5_e-Jb5HQR1rjsk1QA" alt="churn" width="1000" height="300">


**Author**<br>
Kennedy Owino


## Business Problem
---
Telecommunications companies face significant financial losses due to customer churn, wherein customers discontinue their services prematurely. Identifying and predicting customer churn is crucial for mitigating revenue loss, retaining valuable customers, and optimizing marketing strategies. However, without a comprehensive understanding of the underlying patterns and factors contributing to churn, businesses struggle to implement effective retention measures.
Therefore, the business problem of this revolves around the imperative need to identify and predict customer churn accurately.

## Business Understanding
---
In the competitive landscape of telecommunications, customer retention is paramount for sustained profitability and market growth. The ability to predict and prevent customer churn requires a deep understanding of customer behavior, preferences, and dissatisfaction triggers. By leveraging advanced data analytics and machine learning techniques, businesses can uncover actionable insights from vast amounts of customer data to identify at-risk customers and tailor retention strategies accordingly.<br>

### Key Objectives

$i.$ Develop a predictive model to identify customers at risk of churn based on historical data and relevant features.<br>
$ii.$ Identify significant factors influencing customer churn, including service usage patterns, demographics, contract terms, and customer interactions.<br>

$iii.$ Provide actionable insights to decision-makers for proactive churn management and resource allocation.

### Key Stakeholders
$1.$ **Telecom Executives:** Senior management and decision-makers responsible for devising strategic initiatives to improve customer retention and maximize revenue.<br>

$2.$ **Marketing and Sales Teams:** Teams responsible for implementing customer acquisition and retention campaigns, leveraging insights generated by the predictive model to tailor marketing strategies.<br>

$3.$ **Customer Service Representatives:** Frontline staff tasked with addressing customer concerns and enhancing customer satisfaction to reduce churn rates.<br>

$4.$ **Customers:** Ultimately, the end-users whose experiences and interactions drive churn behavior, and who stand to benefit from improved service offerings and personalized retention efforts.



## Data Understanding
---
The dataset utilized in this project is sourced from SyriaTel published on [Kaggle](https://www.kaggle.com/datasets/becksddf/churn-in-telecoms-dataset), encompassing comprehensive customer information. It includes data on customers' state of residence, telephone numbers, and account duration. Additionally, the dataset features columns specifying whether customers have an international plan and voicemail plan, along with details on the number of voicemails received. Furthermore, it encompasses data on the duration of calls, the frequency of calls made, and the corresponding charges incurred during the day, evening, and night periods.

## Exploratory Data Analysis
---